In [8]:
from config_paths import *
import sys
sys.path.append(str(SRC_DIR))

from fuzzy_sql.fuzzy_sql import *
import json
import os
from pathlib import Path

DATASET_NAME='sdgd'

from scipy.stats import gaussian_kde
from scipy.integrate import quad


In [9]:
# set directories
metadata_dir = os.path.join(DATA_DIR, DATASET_NAME,'processed/metadata')
real_dir = os.path.join(DATA_DIR,DATASET_NAME, 'processed/real')
syn_dir = os.path.join(DATA_DIR,DATASET_NAME, 'processed/synthetic')
db_path = os.path.join(DB_DIR, f'{DATASET_NAME}.db')

In [10]:
#Import testing set
with open ('tabular_test_set.json', 'r') as f:
    test_dict=json.load(f)

In [11]:
# connect to db
conn = sqlite3.connect(db_path)

In [12]:
#Slice dictonary for development
selected_datasets=['C1']
test_dict=dict([(key, value) for key,value in test_dict.items() if key in selected_datasets])

In [13]:
#pick any real-syn twin table names for testing
real_name=selected_datasets[0]
syn_name=test_dict[real_name][0]

In [14]:
#import real and syn data 
real_df=pd.read_sql_query(f"SELECT * FROM {real_name}", conn)
syn_df=pd.read_sql_query(f"SELECT * FROM {syn_name}", conn)
with open(os.path.join(metadata_dir,  real_name+'.json'), 'r') as f:
    meta_dict = json.load(f)

In [26]:
query_obj = RndQry(conn, [real_name], [meta_dict])
expr, from_tbl, join_tbl_lst=query_obj.compile_fltr_expr()


In [27]:
expr

"SELECT *  FROM C1  WHERE   native_country NOT LIKE 'United-States' AND  fnlwgt = 544319 "